In [1]:
import numpy as np
import time 

In [2]:
import funciones as func 


In [3]:

Rosembrock      = func.Rosembrock
Rosembrock_Grad = func.grad_Rosembrock
Rosembrock_Hess = func.hess_Rosembrock

Wood            = func.wood_f
Wood_Grad       = func.grad_wood
Wood_Hess       = func.hess_wood

Branin          = func.branin_f
Branin_Grad     = func.grad_branin
Branin_Hess     = func.hess_branin

newton_modificado      = func.newton_modificado



# Step

## Cauchy step

In [4]:
def cauchy_step(g_k,B_k, delta_k):
    '''
    Calcula el paso de cauchy
    Parámetros: 
    ----------------
    
    g_k     : valor del gradiente de la función en la iteración k 
    B_k     : hessiano  de la función en la iteración k 
    delta_k : radio de la región de confianza 
    
    -----------------
    Regresa punto de cauchy p_k_c 
    '''
    
    if g_k.dot(B_k).dot(g_k) <=0:
        tau = 1
        p_k_c = - (delta_k/ np.linalg.norm(g_k))* g_k
        
        return p_k_c
    else: 
        temp  = (np.linalg.norm(g_k))**3 / (delta_k*g_k.dot(B_k).dot(g_k))
        tau   = min(1, temp)
        p_k_c =  -tau* (delta_k/ np.linalg.norm(g_k))* g_k
        
        return  p_k_c 
    

## Dogleg Step

In [5]:
def dogleg_step(g_k, B_k, delta_k):
    '''
    Calcula el tamaño de paso dado el método dogleg 
    Parámetros: 
    ----------------
    
    g_k     : valor del gradiente de la función en la iteración k 
    B_k     : hessiano  de la función en la iteración k 
    delta_k : radio de la región de confianza 
    
    -----------------
    Regresa punto de cauchy p_k_c 
    '''
    
    #tamaño del paso 
    p_k_u = - g_k.dot(g_k)/(g_k.dot(B_k).dot(g_k)) * g_k
    #si b_k es definida positiva 
    p_k_b = - np.linalg.inv(B_k).dot(g_k)
    
    #revisar si el punto está dentro de la región de confianza 
    if np.linalg.norm(p_k_b) <= delta_k :
        p_k = p_k_b 
        
    else:
        if  np.linalg.norm(p_k_u) >= delta_k :
            p_k = cauchy_step(g_k,B_k,delta_k)
        else:
            dif   = p_k_b - p_k_u 
            a     = dif.dot(dif)
            b     = 2.0 * p_k_b.dot(dif)
            c     = p_k_u.dot(p_k_u) - delta_k**2 
            tau_k = 1.0 + (-b+ np.sqrt(b**2 - 4.0 * a * c))/ (2.0*a)
            
            if tau_k <= 1.0: 
                p_k = tau_k * p_k_u 
            else: 
                p_k = p_k_u + (tau_k-1)*dif
   

    return p_k        

## Newton-Cauchy alterno 

In [6]:
def n_c_step(g_k, b_k, delta_k):
    '''
    Función que calcula el paso dado el método de newton-cauchy alterno 
    Parámetros:
    --------------------
    g_k     : gradiente de la función en la iteración k 
    b_k     : hessiano de la función en la iteracción k
    delta_k : región de confianza
    ---------------------
    Regresa : p_k paso en la iteración k 
    '''
    
    #primero obtenemos el paso de newton
    p_k = np.linalg.solve(b_k, -g_k)
    
    if np.linalg.norm(p_k) >= delta_k: 
        p_k = cauchy_step(g_k,b_k,delta_k)
    
    return p_k 

# Método de Newton 

# Implementa el método de Region de Confianza basado en Dogleg.

In [7]:
def trust_region(params = {}, step = 'dogleg'):
    '''
    Calcula un mínimo dado una región de confianza
    Parámetros 
    -----------
    params : Diccionario de parámetros
    x_0      : punto inicial
    f        : función a evaluar
    f_grad   : regresa gradiente de f
    f_hess   : regresa hessiano de f
    max_iter : número maximo de iteraciones

    Trust-Region : Subparámetros del algoritmo región de confianza
      delta_k    : tamaño del radio de la región de confianza
      delta_max  : máxima tolerancia permitida
      eta        : tolerancia para métrica ro
      eta_min    : valor mínimo permitido [1/4, 3/4]
      eta1_hat   : parámetro de cambio eta_1 *delta_k
      eta_max    : valor máximo permitido [1/4, 3/4]
      eta2_hat   : parámetro de cambio  min(eta_2 * delta_k, delta_max)
      step   : método con el que se calcula el paso p_k
  
    Regresa 
    -----------
    f_hist : historial de cambios de f(x_k)
    g_hist : historial de norma del gradiente \nabla f(x_k)
    x_hist : historial de cambios de valores x_k
    '''
    #parámetros 
    x_k         = params['x_0']
    f           = params['f']
    f_grad      = params['f_grad']
    f_hess      = params['f_hess']
    max_iter    = params['max_iter']
    tau_x       = params['tau_x']
    tau_f       = params['tau_f']
    tau_f_grad  = params['tau_grad']
    # Región de confianza
    delta_k     = params['Trust-Region']['delta_k']
    delta_max   = params['Trust-Region']['delta_max']
    eta         = params['Trust-Region']['eta']
    eta_1       = params['Trust-Region']['eta_1'] 
    eta_2       = params['Trust-Region']['eta_2']
    eta1_hat    = params['Trust-Region']['eta1_hat']
    eta2_hat    = params['Trust-Region']['eta2_hat']
    
      # Subpámetros para la función
    if f.__name__ == 'branin_f' :
        sub_params = {
                  'a' : params['a'],
                  'b' : params['b'],
                  'c' : params['c'],
                  'r' : params['r'],
                  's' : params['s'],
                  't' : params['t']
                 }
    else :
        sub_params = {}
    
    f_history  = []
    g_history = []
    x_history = []
    
    #primera iteración 
    f_k      = f(x_k, params = sub_params)
    g_k      = f_grad(x_k, params = sub_params)
    B_k      = f_hess(x_k, params = sub_params)
   
    f_history.append(f_k)
    g_history.append(np.linalg.norm(g_k))
    x_history.append(x_k)
    
    k = 0
    while True: 
        if step == 'dogleg':
            p_k = dogleg_step(g_k, B_k, delta_k)
            
        elif step == 'cauchy':
            p_k = cauchy_step(g_k, B_k, delta_k)
        
        elif step == 'newton-cauchy':
            p_k = n_c_step(g_k,B_k,delta_k)
        else: 
            print("No se recibió método para encontrar el paso")
            return 
        #computamos ro, m_k(0) = f(x_k)
        mk_p = f_k + g_k.dot(p_k) + 0.5*p_k.dot(B_k).dot(p_k)
        ro = (f_k- f(x_k+p_k, params = sub_params))/(f_k-mk_p)
        
        if ro < eta_1 :  #cambiamos el radio de la región de confianza 
            delta_k = delta_k * eta1_hat 
        
        elif  ro >= eta_1: 
            if ro > eta_2 and np.linalg.norm(p_k) == delta_k : 
                delta_k = min(eta2_hat*delta_k, eta)
        
        if ro > eta: #actualizamos el punto 
            x_k    = x_k + p_k
            
            
        #nuevos valores de la función, gradiente y hessiano 
        f_k      = f(x_k, params = sub_params)
        g_k      = f_grad(x_k, params = sub_params)
        b_k      = f_hess(x_k, params = sub_params)
    
        x_history.append(x_k)
        f_history.append(f_k)
        g_history.append(np.linalg.norm(g_k))
        
        # Actualizamos contador
        k   = k + 1
        
        #--------- CRITERIOS DE PARO ----------------#
        if k > max_iter : 
            break
        
        if np.linalg.norm(g_k) < tau_f_grad :
            break  
        
        if np.linalg.norm(x_history[-1] - x_history[-2]) / max(np.linalg.norm(x_history[-2]), 1.0) < tau_x :
              break
        
        if np.abs(f_history[-1] - f_history[-2]) / max(np.abs(f_history[-2]), 1.0) < tau_f :
              break 
    
    return np.array(f_history), np.array(g_history), x_k
  

# Rosembrock

In [8]:
iterations = 30
n = 100

time_dogleg                = []
iterations_dogleg          = []

time_newton_cauchy         = []
iterations_newton_cauchy   = []

time_newton_modified       = []
iterations_newton_modified = []

for i in range(iterations) :
    print('Corrida :', i+1)
    x_0 = [1.0 + np.random.uniform(-2,2) for i in range(n)]
    x_0 = np.array(x_0)

    params = {
          'x_0'      : x_0,
          'f'        : Rosembrock,
          'f_grad'   : Rosembrock_Grad,
          'f_hess'   : Rosembrock_Hess,
          'max_iter' : 1000,
          'tau_x'    : 1e-12,
          'tau_f'    : 1e-12,
          'tau_grad' : 1e-12,
          'Trust-Region' : {
                            'delta_k'   : 0.1,
                            'delta_max' : 0.2,
                            'eta'       : 0.1,
                            'eta_1'     : 0.25,
                            'eta_2'     : 0.75,
                            'eta1_hat'  : 0.25,
                            'eta2_hat'  : 2.0
                           }
          }
  # DOGLEG
    star              = time.time()
    f_hist, g_hist, x = trust_region(params, step = 'dogleg')

    ex_time           = time.time() - star  

    time_dogleg.append(ex_time)
    iterations_dogleg.append(f_hist.shape[0])

  # NEWTON-CAUCHY
    star              = time.time()
    f_hist, g_hist, x = trust_region(params, step = 'newton-cauchy')

    ex_time           = time.time() - star  

    time_newton_cauchy.append(ex_time)
    iterations_newton_cauchy.append(f_hist.shape[0])
    
    params = {
              'x_0'      : x_0,
              'f'        : Rosembrock,
              'f_grad'   : Rosembrock_Grad,
              'f_hess'   : Rosembrock_Hess,
              'max_iter' : 10000,
              'tau_x'    : 1e-12,
              'tau_f'    : 1e-12,
              'tau_grad' : 1e-12,
              'beta'     : 1e-12,
              'cholesky' : {
                            'max_iter' : 100
                            }
            }
    star                = time.time()
    f_hist, g_hist, x_k = newton_modificado(params)
    ex_time             = time.time() - star  

    time_newton_modified.append(ex_time)
    iterations_newton_modified.append(f_hist.shape[0])

print('Dogleg:')
print('Tiempo promedio: ', np.mean(time_dogleg), ' Promedio de iteraciones : ', np.mean(iterations_dogleg))
print('Newton-Cauchy:')
print('Tiempo promedio: ', np.mean(time_newton_cauchy), ' Promedio de iteraciones : ', np.mean(iterations_newton_cauchy))
print('Newton Modificado:')
print('Tiempo promedio: ', np.mean(time_newton_modified), ' Promedio de iteraciones : ', np.mean(iterations_newton_modified))

Corrida : 1
Corrida : 2
Corrida : 3
Corrida : 4
Corrida : 5
Corrida : 6
Corrida : 7
Corrida : 8
Corrida : 9
Corrida : 10
Corrida : 11
Corrida : 12
Corrida : 13
Corrida : 14
Corrida : 15
Corrida : 16
Corrida : 17
Corrida : 18
Corrida : 19
Corrida : 20
Corrida : 21
Corrida : 22
Corrida : 23
Corrida : 24
Corrida : 25
Corrida : 26
Corrida : 27
Corrida : 28
Corrida : 29
Corrida : 30
Dogleg:
Tiempo promedio:  0.6255225817362468  Promedio de iteraciones :  336.5
Newton-Cauchy:
Tiempo promedio:  0.5805398225784302  Promedio de iteraciones :  344.8
Newton Modificado:
Tiempo promedio:  0.3442240079243978  Promedio de iteraciones :  34.1


# Wood 

In [12]:
iterations = 30
n = 4

time_dogleg                = []
iterations_dogleg          = []

time_newton_cauchy         = []
iterations_newton_cauchy   = []

time_newton_modified       = []
iterations_newton_modified = []

for i in range(iterations) :
    print('Corrida :', i+1)
    x_0 = [1.0 + np.random.uniform(-2,2) for i in range(n)]
    x_0 = np.array(x_0)

    params = {
          'x_0'      : x_0,
          'f'        : Wood,
          'f_grad'   : Wood_Grad,
          'f_hess'   : Wood_Hess,
          'max_iter' : 1000,
          'tau_x'    : 1e-12,
          'tau_f'    : 1e-12,
          'tau_grad' : 1e-12,
          'Trust-Region' : {
                            'delta_k'   : 0.1,
                            'delta_max' : 0.2,
                            'eta'       : 0.1,
                            'eta_1'     : 0.25,
                            'eta_2'     : 0.75,
                            'eta1_hat'  : 0.25,
                            'eta2_hat'  : 2.0
                           }
          }
  # DOGLEG
    star              = time.time()
    f_hist, g_hist, x = trust_region(params, step = 'dogleg')

    ex_time           = time.time() - star  
    print(x)
    time_dogleg.append(ex_time)
    iterations_dogleg.append(f_hist.shape[0])

  # NEWTON-CAUCHY
    star              = time.time()
    f_hist, g_hist, x = trust_region(params, step = 'newton-cauchy')
    print(x)
    ex_time           = time.time() - star  

    time_newton_cauchy.append(ex_time)
    iterations_newton_cauchy.append(f_hist.shape[0])
    
    params = {
              'x_0'      : x_0,
              'f'        : Wood,
              'f_grad'   : Wood_Grad,
              'f_hess'   : Wood_Hess,
              'max_iter' : 10000,
              'tau_x'    : 1e-12,
              'tau_f'    : 1e-12,
              'tau_grad' : 1e-12,
              'beta'     : 1e-12,
              'cholesky' : {
                            'max_iter' : 100
                            }
            }
    star                = time.time()
    f_hist, g_hist, x_k = newton_modificado(params)
    ex_time             = time.time() - star  
    print(x_k)
    time_newton_modified.append(ex_time)
    iterations_newton_modified.append(f_hist.shape[0])

print('Dogleg:')
print('Tiempo promedio: ', np.mean(time_dogleg), ' Promedio de iteraciones : ', np.mean(iterations_dogleg))
print('Newton-Cauchy:')
print('Tiempo promedio: ', np.mean(time_newton_cauchy), ' Promedio de iteraciones : ', np.mean(iterations_newton_cauchy))
print('Newton Modificado:')
print('Tiempo promedio: ', np.mean(time_newton_modified), ' Promedio de iteraciones : ', np.mean(iterations_newton_modified))


Corrida : 1
[1.01664471 1.06669602 0.59108372 0.41803206]
[0.78013404 0.58073452 1.06255704 1.16289013]
[0.99999933 0.99999865 1.0000007  1.00000141]
Corrida : 2
[0.34811173 0.11883468 1.35988917 1.85079055]
[0.3490105  0.11890745 1.35971882 1.85034408]
[0.99999934 0.99999868 1.00000069 1.00000138]
Corrida : 3
[0.98051264 1.01518095 0.48863214 0.28204118]
[0.97972869 1.0134879  0.48828216 0.28185624]
[1.00000067 1.00000134 0.9999993  0.9999986 ]
Corrida : 4
[1.01773272 1.03567631 0.98135597 0.96335362]
[1.03953833 1.08078429 0.95886933 0.9192802 ]
[0.99999934 0.99999868 1.00000069 1.00000138]
Corrida : 5
[1.22456009 1.5005852  0.69424233 0.48459577]
[ 1.36282928  1.8681111  -0.29960473  0.09358888]
[1.00000066 1.00000132 0.99999931 0.99999862]
Corrida : 6
[ 1.28471301  1.65237229 -0.61376434  0.3454484 ]
[ 1.2782968   1.63500157 -0.58624068  0.32962693]
[1.00000067 1.00000135 0.9999993  0.99999859]
Corrida : 7
[1.02359935 1.10471537 0.72863681 0.6002192 ]
[1.06671456 1.1416095  0.79399

# Branin

In [10]:
iterations = 30


time_dogleg                = []
iterations_dogleg          = []

time_newton_cauchy         = []
iterations_newton_cauchy   = []

time_newton_modified       = []
iterations_newton_modified = []

for i in range(iterations) :
    print('Corrida :', i+1)
    x_0 = np.array((np.pi, 2.275))
    x_0 = x_0 + np.random.uniform(-2,2) 


    params = {
          'x_0'      : x_0,
          'f'        : Branin,
          'f_grad'   : Branin_Grad,
          'f_hess'   : Branin_Hess,
          'max_iter' : 1000,
          'tau_x'    : 1e-12,
          'tau_f'    : 1e-12,
          'tau_grad' : 1e-12,
          'a' : 1.0,
          'b' : 5.1 / (4.0 * np.pi**2),
          'c' : 5.0 / np.pi,
          'r' : 6.0,
          's' : 10.0,
          't' : 1.0 / (8.0 * np.pi),
          'Trust-Region' : {
                            'delta_k'   : 0.1,
                            'delta_max' : 0.2,
                            'eta'       : 0.1,
                            'eta_1'     : 0.25,
                            'eta_2'     : 0.75,
                            'eta1_hat'  : 0.25,
                            'eta2_hat'  : 2.0
                           }
          }
  # DOGLEG
    star              = time.time()
    f_hist, g_hist, x = trust_region(params, step = 'dogleg')

    ex_time           = time.time() - star  

    time_dogleg.append(ex_time)
    iterations_dogleg.append(f_hist.shape[0])

  # NEWTON-CAUCHY
    star              = time.time()
    f_hist, g_hist, x = trust_region(params, step = 'newton-cauchy')

    ex_time           = time.time() - star  

    time_newton_cauchy.append(ex_time)
    iterations_newton_cauchy.append(f_hist.shape[0])
    
    params = {
              'x_0'      : x_0,
              'f'        : Branin,
              'f_grad'   : Branin_Grad,
              'f_hess'   : Branin_Hess,
              'max_iter' : 10000,
              'tau_x'    : 1e-12,
              'tau_f'    : 1e-12,
              'tau_grad' : 1e-12,
              'beta'     : 1e-12,
              'a' : 1.0,
              'b' : 5.1 / (4.0 * np.pi**2),
              'c' : 5.0 / np.pi,
              'r' : 6.0,
              's' : 10.0,
              't' : 1.0 / (8.0 * np.pi),
              'cholesky' : {
                            'max_iter' : 100
                            }
            }
    star                = time.time()
    f_hist, g_hist, x_k = newton_modificado(params)
    ex_time             = time.time() - star  

    time_newton_modified.append(ex_time)
    iterations_newton_modified.append(f_hist.shape[0])

print('Dogleg:')
print('Tiempo promedio: ', np.mean(time_dogleg), ' Promedio de iteraciones : ', np.mean(iterations_dogleg))
print('Newton-Cauchy:')
print('Tiempo promedio: ', np.mean(time_newton_cauchy), ' Promedio de iteraciones : ', np.mean(iterations_newton_cauchy))
print('Newton Modificado:')
print('Tiempo promedio: ', np.mean(time_newton_modified), ' Promedio de iteraciones : ', np.mean(iterations_newton_modified))

Corrida : 1
Corrida : 2
Corrida : 3
Corrida : 4
Corrida : 5
Corrida : 6
Corrida : 7
Corrida : 8
Corrida : 9
Corrida : 10
Corrida : 11
Corrida : 12
Corrida : 13
Corrida : 14
Corrida : 15
Corrida : 16
Corrida : 17
Corrida : 18
Corrida : 19
Corrida : 20
Corrida : 21
Corrida : 22
Corrida : 23
Corrida : 24
Corrida : 25
Corrida : 26
Corrida : 27
Corrida : 28
Corrida : 29
Corrida : 30
Dogleg:
Tiempo promedio:  0.003046751022338867  Promedio de iteraciones :  23.333333333333332
Newton-Cauchy:
Tiempo promedio:  0.002301025390625  Promedio de iteraciones :  23.566666666666666
Newton Modificado:
Tiempo promedio:  0.004870764414469401  Promedio de iteraciones :  25.966666666666665
